# Plots

## Imports and file management

In [19]:
# Importing libraries
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import os
import re
import shutil

plt.style.use("science.mplstyle")

Cleaning: all *.dat files, inside output file

In [20]:
allfiles = sorted(glob.glob("*.dat"))
if not allfiles:
    print("No '*.dat' found")

outdir_data = "outputs"
os.makedirs(outdir_data, exist_ok=True)

moved_files = []
for f in allfiles:
    dst = os.path.join(outdir_data, os.path.basename(f))
    shutil.move(f, dst)             
    moved_files.append(dst)

print(f"Moving {allfiles}")

No '*.dat' found
Moving []


The timeseries files, for the plotting

In [21]:
pattern = os.path.join(outdir_data, "timeseries_L*_T*_MCSTOT*.dat")
files = sorted(glob.glob(pattern))
if not files:
    raise SystemExit(f"No '{pattern}' found")

print(f"Reading {files}")

Reading ['outputs/timeseries_L100_T2.00_MCSTOT100000000.dat', 'outputs/timeseries_L100_T2.10_MCSTOT100000000.dat', 'outputs/timeseries_L100_T2.10_MCSTOT108.dat', 'outputs/timeseries_L100_T2.20_MCSTOT100000000.dat', 'outputs/timeseries_L100_T2.27_MCSTOT100000000.dat', 'outputs/timeseries_L100_T2.30_MCSTOT100000000.dat', 'outputs/timeseries_L100_T2.40_MCSTOT100000000.dat', 'outputs/timeseries_L100_T2.50_MCSTOT100000000.dat', 'outputs/timeseries_L100_T2.60_MCSTOT100000000.dat', 'outputs/timeseries_L100_T2.70_MCSTOT100000000.dat', 'outputs/timeseries_L100_T2.70_MCSTOT108.dat', 'outputs/timeseries_L100_T2.80_MCSTOT100000000.dat', 'outputs/timeseries_L100_T2.90_MCSTOT100000000.dat', 'outputs/timeseries_L100_T3.00_MCSTOT100000000.dat', 'outputs/timeseries_L20_T2.00_MCSTOT1000000.dat', 'outputs/timeseries_L20_T2.00_MCSTOT100000000.dat', 'outputs/timeseries_L20_T2.10_MCSTOT108.dat', 'outputs/timeseries_L20_T2.70_MCSTOT108.dat', 'outputs/timeseries_L20_T2.80_MCSTOT108.dat', 'outputs/timeseries_L

Plots E and M (not |M|) as a function of MCS, for:
- L     = 100
- nmeas = 10
- nMCS  = 10^8
- T     = 2.0, 2.1, 2.2, ..., 3.0

## What is asked in 4.

In [ ]:
# Compute <E>/L^2 (= <E/N>) by summing E/N from MCS > 1000 and dividing by Nsamples

burnin_mcs = 1000
sum_E_per_spin = 0.0
nsamples = 0

with open("outputs/timeseries_L20_T2.00_MCSTOT100000000.dat", "r", encoding="latin1") as f:
    next(f, None) 
    next(f, None)  # skip header

    for line in f:
        s = line.strip()
        if (not s) or s.startswith("#"):
            continue

        parts = s.split()
        if len(parts) < 2:
            continue

        try:
            mcs = int(float(parts[0]))
            e_over_L2 = float(parts[1])   # E/N = E/L^2
        except ValueError:
            continue

        if mcs <= burnin_mcs:
            continue

        sum_E_per_spin += e_over_L2
        nsamples += 1

Ebar_over_L2 = sum_E_per_spin / nsamples   # <E>/L^2 = <E/N> 
print("Samples:", nsamples)
print("<E>/L^2 =", Ebar_over_L2)


Samples: 9999898
<E>/L^2 = -1.7455883709978008


In [30]:
# Plot sigma E as function of m, for L=20, T=2.0

outdir_binning = os.path.join(outdir_data, "plots")
os.makedirs(outdir_binning, exist_ok=True)

# The asked files
bfiles = [
    os.path.join(outdir_data, "binning_program_L_20_T_2.0.dat"),
]
bfiles = [f for f in bfiles if os.path.exists(f)]
print("Binning files:", bfiles)

# ----------------------------
# 7) sigma_E vs m  (NO zeros)
# ----------------------------
plt.figure()
for f in bfiles:
    data = np.loadtxt(f, ndmin=2)
    if data.size == 0 or data.shape[1] < 5:
        print(f"Skipping empty/bad file: {f}")
        continue

    bb    = data[:, 0]
    sig_E = data[:, 2]
    mask = sig_E > 0

    base = os.path.basename(f)
    T = base.split("_T_")[-1].replace(".dat", "")
    plt.plot(bb[mask], sig_E[mask], marker="o", linestyle="-", label=f"T={T}")

plt.xscale("log")
plt.yscale("log")
plt.xlabel("Block size m")
plt.ylabel(r"$\sigma_E$")
plt.legend(prop={'size': 6})
plt.savefig(os.path.join(outdir_binning, "sigmaE_L20_T2_T2p27_T2p6.pdf"), bbox_inches="tight")
plt.close()

Binning files: ['outputs/binning_program_L_20_T_2.0.dat']


## What is asked in 6.

In [4]:
# Directory for plots
outdir = os.path.join(outdir_data, "plots")
os.makedirs(outdir, exist_ok=True)

# Regex to parse filenames like: timeseries_L20_T2.00_MCSTOT100000000.dat (ChatGPT)
#timeseries_L20_T2.00_MCSTOT1000000
#pat = re.compile(r"timeseries_L(?P<L>\d+)_T(?P<T>\d+(?:\.\d+)?)_MCSTOT(?P<MC>\d+)\.dat")
pat = re.compile(r"^timeseries_L(?P<L>\d+)_T(?P<T>\d+(?:\.\d+)?)_MCSTOT(?P<MC>\d+)\.dat$")


# sample  from 1e3 to 1e6: one point every 100 data (ChatGPT)
# Sample every 1000 up to 1e8
def sample_point6(mcs, y):
    mask_1e3 = (mcs <= 10**3)
    mask_1e6 = (mcs > 10**3) & (mcs <= 10**6)
    mask_1e8 = (mcs > 10**6) & (mcs <= 10**8)

    x1, y1 = mcs[mask_1e3], y[mask_1e3]
    x2, y2 = mcs[mask_1e6][::100], y[mask_1e6][::100]
    x3, y3 = mcs[mask_1e8][::1000], y[mask_1e8][::1000]
    return np.concatenate([x1, x2, x3]), np.concatenate([y1, y2, y3]) 

# Plot asked T's
Temps = {"2.00", "2.27", "2.60"}  

# ENERGIES
plt.figure()
for dades in files:
    base = os.path.basename(dades)
    m = pat.match(base)
    if not m:
        continue

    L = m.group("L")
    T = m.group("T")

    if L != "100" or T not in Temps:
        continue

    data = np.loadtxt(dades, comments="#", ndmin=2)
    if data.size == 0 or data.shape[1] < 3:
        print(f"Skipping empty/bad file: {dades}")
        continue

    mcs = data[:, 0]
    E   = data[:, 1]

    xs, Es = sample_point6(mcs, E)
    plt.plot(xs, Es, linestyle="-", marker="o", markersize=2, label=f"T={T}") 

plt.xlabel("MC time")
plt.ylabel("Energy (E/N)")
plt.xscale("log")
plt.legend(prop={'size': 6})
plt.savefig(os.path.join(outdir, "E_L100_T2p00_T2p27_T2p60_upto1e8.pdf"), bbox_inches="tight")  # pdf vector
plt.close()

# MAGNETIZATIONS
plt.figure()
for dades in files:
    base = os.path.basename(dades)
    m = pat.match(base)
    if not m:
        continue

    L = m.group("L")
    T = m.group("T")

    if L != "100" or T not in Temps:
        continue

    data = np.loadtxt(dades, comments="#", ndmin=2)
    if data.size == 0 or data.shape[1] < 3:
        print(f"Skipping empty/bad file: {dades}")
        continue

    mcs = data[:, 0]
    M   = data[:, 2]  

    xs, Ms = sample_point6(mcs, M)
    plt.plot(xs, Ms, linestyle="-", marker="o", markersize=2, label=f"T={T}") 

plt.xlabel("MC time")
plt.ylabel("Magnetization (M/N)")
plt.xscale("log")
plt.legend(prop={'size': 6})
plt.savefig(os.path.join(outdir, "M_L100_T2p00_T2p27_T2p60_upto1e8.pdf"), bbox_inches="tight")  
plt.close()

# COMPARE L=20, L=100
# Sample every 100 up to 1e6 (ChatGPT)
# Sample every 1000 up to 1e8
def sample_every_100_upto_1e6(mcs, y):
    mask_1e3 = (mcs <= 10**3)
    mask_1e6 = (mcs > 10**3) & (mcs <= 10**6)
    mask_1e8 = (mcs > 10**6) & (mcs <= 10**8)

    x1, y1 = mcs[mask_1e3], y[mask_1e3]
    x2, y2 = mcs[mask_1e6][::100], y[mask_1e6][::100]
    x3, y3 = mcs[mask_1e8][::1000], y[mask_1e8][::1000]

    return np.concatenate([x1, x2, x3]), np.concatenate([y1, y2, y3])


plt.figure()
for dades in files:
    if not os.path.exists(dades):
        print(f"Missing file (skipping): {dades}")
        continue

    base = os.path.basename(dades)
    m = pat.match(base)
    if not m:
        continue

    L  = m.group("L")
    T  = m.group("T")
    MC = m.group("MC")   

    if T != "2.00" or L not in {"20", "100"}:
        continue

    # both cases to compare
    if (L == "20"  and MC != "1000000") or (L == "100" and MC != "100000000"):
        continue

    data = np.loadtxt(dades, comments="#", ndmin=2)
    if data.size == 0 or data.shape[1] < 3:
        print(f"Skipping empty/bad file: {dades}")
        continue

    mcs = data[:, 0]
    M   = data[:, 2]  

    xs, Ms = sample_every_100_upto_1e6(mcs, M)
    plt.plot(xs, Ms, linestyle="-", marker="o", markersize=2, label=f"L={L}, T={T}")

plt.xlabel("MC time")
plt.ylabel("Magnetization (M/N)")
plt.xscale("log")
plt.legend(prop={'size': 6})
plt.savefig(os.path.join(outdir, "M_T2p00_L20_vs_L100_upto1e8_step100.pdf"), bbox_inches="tight")
plt.close()


print(f"Plots saved at: {outdir}/")


Plots saved at: outputs/plots/


In [5]:
try:
    data = np.loadtxt(dades, comments="#", ndmin=2)
except UnicodeDecodeError as e:
    print("Bad encoding in:", dades)
    raise


/var/folders/q8/ht1hgr5d6d1bwfgb2rnkpttw0000gp/T/ipykernel_19598/942269625.py:2: UserWarning: loadtxt: input contained no data: "outputs/timeseries_L20_T2.90_MCSTOT108.dat"
  data = np.loadtxt(dades, comments="#", ndmin=2)


## What is asked in 7.

In [15]:
outdir_binning = os.path.join(outdir_data, "plots")
os.makedirs(outdir_binning, exist_ok=True)

# The asked files
bfiles = [
    os.path.join(outdir_data, "binning_program_L_100_T_2.dat"),
    os.path.join(outdir_data, "binning_program_L_100_T_2.27.dat"),
    os.path.join(outdir_data, "binning_program_L_100_T_2.6.dat"),
]
bfiles = [f for f in bfiles if os.path.exists(f)]
print("Binning files:", bfiles)

# ----------------------------
# 7) sigma_E vs m  (NO zeros)
# ----------------------------
plt.figure()
for f in bfiles:
    data = np.loadtxt(f, ndmin=2)
    if data.size == 0 or data.shape[1] < 5:
        print(f"Skipping empty/bad file: {f}")
        continue

    bb    = data[:, 0]
    sig_E = data[:, 2]
    mask = sig_E > 0

    base = os.path.basename(f)
    T = base.split("_T_")[-1].replace(".dat", "")
    plt.plot(bb[mask], sig_E[mask], marker="o", linestyle="-", label=f"T={T}")

plt.xscale("log")
plt.yscale("log")
plt.xlabel("Block size m")
plt.ylabel(r"$\sigma_E$")
plt.legend(prop={'size': 6})
plt.savefig(os.path.join(outdir_binning, "sigmaE_L100_T2_T2p27_T2p6.pdf"), bbox_inches="tight")
plt.close()

# ----------------------------
# 7) sigma_M vs m  (NO zeros)
# ----------------------------
plt.figure()
for f in bfiles:
    data = np.loadtxt(f, ndmin=2)
    if data.size == 0 or data.shape[1] < 5:
        print(f"Skipping empty/bad file: {f}")
        continue

    bb    = data[:, 0]
    sig_M = data[:, 4]
    mask = sig_M > 0

    base = os.path.basename(f)
    T = base.split("_T_")[-1].replace(".dat", "")
    plt.plot(bb[mask], sig_M[mask], marker="o", linestyle="-", label=f"T={T}")

plt.xscale("log")
plt.yscale("log")
plt.xlabel("Block size m")
plt.ylabel(r"$\sigma_M$")
plt.legend(prop={'size': 6})
plt.savefig(os.path.join(outdir_binning, "sigmaM_L100_T2_T2p27_T2p6.pdf"), bbox_inches="tight")
plt.close()

print(f"Binning plots saved to: {outdir_binning}/")


Binning files: ['outputs/binning_program_L_100_T_2.dat', 'outputs/binning_program_L_100_T_2.27.dat', 'outputs/binning_program_L_100_T_2.6.dat']
Binning plots saved to: outputs/plots/


## What is asked in 8.

In [10]:
# <E>/N and <|M|>/N vs T with shaded statistical error band
# (all temperatures; highlight T=2.00, 2.27, 2.60 with filled markers)

# <E>/N and <|M|>/N vs T with shaded statistical error band
# (all temperatures; highlight T=2.00, 2.27, 2.60 with filled markers)

# ----------------------------
# <E>/N vs T (band) -- red (POINTS ONLY)
# ----------------------------
plt.figure()

cE = "#FF2C00"  # red

plt.fill_between(T, E - Eerr, E + Eerr, color=cE, alpha=0.5, linewidth=0)

# non-critical
plt.plot(
    T[~is_crit], E[~is_crit],
    linestyle="None", marker="x", ms=ms_all,
    markerfacecolor="none", markeredgecolor=cE,
    label=r"$\langle E\rangle/N$"
)

# critical
plt.plot(
    T[is_crit], E[is_crit],
    linestyle="None", marker="o", ms=ms_all,
    color=cE
)

plt.xlabel("Temperature T")
plt.ylabel(r"$\langle E\rangle/N$")
plt.legend(prop={'size': 6})
plt.savefig(os.path.join(outdir_binning, "E_vs_T_all_band.pdf"), bbox_inches="tight")
plt.close()


# ----------------------------
# <|M|>/N vs T (band) -- violet (POINTS ONLY)
# ----------------------------
plt.figure()

cM = "#845B97"  # violet

plt.fill_between(T, M - Merr, M + Merr, color=cM, alpha=0.5, linewidth=0)

# non-critical
plt.plot(
    T[~is_crit], M[~is_crit],
    linestyle="None", marker="x", ms=ms_all,
    markerfacecolor="none", markeredgecolor=cM,
    label=r"$\langle |M|\rangle/N$"
)

# critical
plt.plot(
    T[is_crit], M[is_crit],
    linestyle="None", marker="o", ms=ms_all,
    color=cM
)

plt.xlabel("Temperature T")
plt.ylabel(r"$\langle |M|\rangle/N$")
plt.legend(prop={'size': 6})
plt.savefig(os.path.join(outdir_binning, "absM_vs_T_all_band.pdf"), bbox_inches="tight")
plt.close()


NameError: name 'Eerr' is not defined

<Figure size 350x262.5 with 0 Axes>